In [1]:
import pymongo

In [2]:
mongo_db = {
    'HOSTNAME': '127.0.0.1:27017',
    'USERNAME': 'webscrapy',
    'PASSWORD': '68f539388f66a374908f3df559eb4ea2',
    'DATABASE': 'realestate'
}

In [3]:
connection = pymongo.MongoClient(host=mongo_db['HOSTNAME'],
                                      username=mongo_db['USERNAME'],
                                      password=mongo_db['PASSWORD'],
                                      authSource=mongo_db['DATABASE'],
                                      authMechanism='SCRAM-SHA-1')
db = connection[mongo_db['DATABASE']]

In [4]:
collections = ['raw_alonhadat_news', 'raw_bds_news', 'raw_chotot_news']

In [1]:
def normalize_text(raw_description: str, annotator) -> str:
    # remove strip_emoji
    import re
    def strip_emoji(text):
        RE_EMOJI = re.compile(u'([\U00002600-\U000027BF])|([\U0001f300-\U0001f64F])|([\U0001f680-\U0001f6FF])')
        return RE_EMOJI.sub(r'', text)

    def normalize_line(line: str) -> str:
        line = line.strip(' ')
        line = line.strip(' -+')

        return line

    def segment_word(line: str, annotator) -> str:
        segmented_list = annotator.tokenize(line)
        segmented_line = ''
        for sen in segmented_list:
            segmented_line += ' '.join(sen)
            segmented_line += ' '

        segmented_line = segmented_line.replace(' .', '.')
        segmented_line = segmented_line.replace(' ,', ',')
        segmented_line = segmented_line.replace(' / ', '/')
        segmented_line = segmented_line.replace(' :', ':')

        return segmented_line

    raw_text: str = strip_emoji(raw_description)
    raw_text = raw_text.replace('\r', '')
    raw_text = raw_text.replace('"', '')

    lines = raw_text.split('\n')
    lines = [normalize_line(line) for line in lines]

    segmented_lines = []
    for line in lines:
        if len(line) and line != '':
            segmented_lines.append(segment_word(line, annotator=annotator))
        else:
            segmented_lines.append(line)

    flag = False
    cleand_lines = []

    for i in range(len(segmented_lines) - 1):
        line1 = segmented_lines[i]
        line2 = segmented_lines[i+1]

        if line1 == line2 and line1 == '':
            flag = True
            continue
        else:
            flag = False

        if not flag:
            cleand_lines.append(line1)

    cleand_lines.append(segmented_lines[-1])
    cleaned_text = '\n'.join(cleand_lines)


    return cleaned_text

In [6]:
from vncorenlp import VnCoreNLP
annotator = VnCoreNLP(address="http://127.0.0.1", port=9000)

In [13]:
import json

count = 0
for item in db[collections[2]].find({}, {"description": 1}):
    with open('dataset_ner.json', 'a') as file:

        try:
            raw_description = item['description']
            if raw_description is not None:
                count += 1
                id = str(item['_id'])
                text = normalize_text(item['description'], annotator=annotator)
                # print(text)
                # print("==" * 20)
                # if count >= 100:
                #     break
                record = {'id': id, 'text': text}
                file.write(json.dumps(record))
                file.write('\n')

        except:
            count += 1
            print(item)

In [41]:
from vncorenlp import VnCoreNLP
annotator = VnCoreNLP(address="http://127.0.0.1", port=9000)

In [46]:
text = "\n"

# To perform word segmentation, POS tagging, NER and then dependency parsing
annotated_text = annotator.annotate(text)

# To perform word segmentation only
word_segmented_text = annotator.tokenize(text)

In [47]:
word_segmented_text

[]